# Imports

In [1]:
%load_ext autoreload
%autoreload 2

import os

import pandas as pd

# Data loading

In [11]:
stocks_dict

{'adnc': {'stf_spectogram':                 coef_0             coef_1             coef_2  \
  0     2.0498 + 0.0000i   5.5527 + 0.0000i   4.1448 + 0.0000i   
  1     0.5915 - 1.6932i   1.4713 - 4.9145i   2.3026 - 3.0329i   
  2    -0.8871 - 0.7559i  -3.3730 - 2.2295i  -0.6850 - 2.8983i   
  3    -0.4736 + 0.1602i  -2.1189 + 1.7002i  -1.7762 - 1.0563i   
  4    -0.1729 + 0.0869i   0.4498 + 1.4986i  -1.3776 + 0.2776i   
  ..                 ...                ...                ...   
  124  -0.0065 - 0.0104i   0.0431 + 0.0010i   0.0152 + 0.0488i   
  125  -0.0075 + 0.0097i   0.0284 - 0.0283i   0.0557 + 0.0118i   
  126   0.0097 + 0.0039i   0.0031 - 0.0378i   0.0367 - 0.0415i   
  127   0.0011 - 0.0092i  -0.0252 - 0.0313i  -0.0180 - 0.0450i   
  128  -0.0091 + 0.0000i  -0.0421 + 0.0000i  -0.0444 + 0.0000i   
  
                         coef_3                     coef_4             coef_5  \
  0            1.7926 + 0.0000i           1.7292 + 0.0000i   1.7463 + 0.0000i   
  1            0.